In [0]:
# Import necessary libraries
import time

# Create a dataframe to test a heavy query 
heavy_df_test = spark.sql("""
SELECT user_id, 
       COUNT(*) as total_events,
       COUNT(DISTINCT user_session) as unique_sessions,
       COUNT(DISTINCT product_id) as unique_products,
       AVG(price) as average_price,
       SUM(price) as total_price
FROM oct_events_delta
GROUP BY user_id
ORDER BY total_price
""")

In [0]:
# Note the start time
start = time.time()

# Run the query
heavy_df_test.show()

# Note the end time
end = time.time()

# Store time taken
no_autocache = end - start

print("Execution time:", end - start)

+---------+------------+---------------+---------------+-------------+-----------+
|  user_id|total_events|unique_sessions|unique_products|average_price|total_price|
+---------+------------+---------------+---------------+-------------+-----------+
|555652395|           3|              1|              1|          0.0|        0.0|
|560311797|           3|              1|              1|          0.0|        0.0|
|555698958|           1|              1|              1|          0.0|        0.0|
|559411341|           1|              1|              1|          0.0|        0.0|
|549659950|           2|              2|              1|          0.0|        0.0|
|565444856|           1|              1|              1|          0.0|        0.0|
|560993983|           1|              1|              1|          0.0|        0.0|
|534857072|           2|              1|              1|          0.0|        0.0|
|555570726|           1|              1|              1|          0.0|        0.0|
|506

As .cache() is not supported in community edition. We test autocache by repeating the query execution and check the .explain(True).

In [0]:
# Note the start time
start = time.time()

# Run the query
heavy_df_test.show()

# Note the end time
end = time.time()

# Store time taken
with_autocache = end - start

print("Execution time:", end - start)

+---------+------------+---------------+---------------+-------------+-----------+
|  user_id|total_events|unique_sessions|unique_products|average_price|total_price|
+---------+------------+---------------+---------------+-------------+-----------+
|555652395|           3|              1|              1|          0.0|        0.0|
|560311797|           3|              1|              1|          0.0|        0.0|
|555698958|           1|              1|              1|          0.0|        0.0|
|559411341|           1|              1|              1|          0.0|        0.0|
|549659950|           2|              2|              1|          0.0|        0.0|
|565444856|           1|              1|              1|          0.0|        0.0|
|560993983|           1|              1|              1|          0.0|        0.0|
|534857072|           2|              1|              1|          0.0|        0.0|
|555570726|           1|              1|              1|          0.0|        0.0|
|506

In [0]:
print("Without autocache: ", no_autocache)
print("With autocache: ", with_autocache)
print("Difference: ", no_autocache - with_autocache)
print(f"Autocache is: {(no_autocache / with_autocache):.2f}x faster")


Without autocache:  5.343674421310425
With autocache:  4.598085403442383
Difference:  0.745589017868042
Autocache is: 1.16x faster


In [0]:
# Explain the query
heavy_df_test.explain(True)

== Parsed Logical Plan ==
'Sort ['total_price ASC NULLS FIRST], true
+- 'Aggregate ['user_id], ['user_id, 'COUNT(1) AS total_events#17544, 'COUNT(distinct 'user_session) AS unique_sessions#17545, 'COUNT(distinct 'product_id) AS unique_products#17546, 'AVG('price) AS average_price#17547, 'SUM('price) AS total_price#17548]
   +- 'UnresolvedRelation [oct_events_delta], [], false

== Analyzed Logical Plan ==
user_id: int, total_events: bigint, unique_sessions: bigint, unique_products: bigint, average_price: double, total_price: double
Sort [total_price#17548 ASC NULLS FIRST], true
+- Aggregate [user_id#18517], [user_id#18517, count(1) AS total_events#17544L, count(distinct user_session#18518) AS unique_sessions#17545L, count(distinct product_id#18512) AS unique_products#17546L, avg(price#18516) AS average_price#17547, sum(price#18516) AS total_price#17548]
   +- SubqueryAlias workspace.default.oct_events_delta
      +- Relation workspace.default.oct_events_delta[event_time#18510,event_type